## Imports

In [15]:
import os, shutil
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from matplotlib import pyplot as plt
from datetime import datetime

#turn off annoying warnigns
pd.options.mode.chained_assignment = None  # default='warn'

## Task 1: Data Collection

In [16]:
url = "https://www.worldometers.info/coronavirus/"
htmlContent = requests.get(url).text

soup = BeautifulSoup(htmlContent, 'html.parser')
covidTable = soup.find("table",attrs={"id": "main_table_countries_today"})

## Task 2: Main Data Extract

In [17]:
# extract the table headings from the soup
head = covidTable.thead.find_all("tr")
headings = []

for th in head[0].find_all("th"):
    headings.append(th.text.replace("\n", "").strip())

# extract actual data from the soup
body = covidTable.tbody.find_all("tr")
data = []

# iterate through every row in the html
for r in range(1,len(body)):
    row = []
    # find all column entries in that particular row
    for tr in body[r].find_all("td"):
        row.append(tr.text.replace("\n","").strip())
    len(row)
    data.append(row)

df = pd.DataFrame(data,columns = headings)

######################################################################
#                         CLEAN DATA TABLE                           #
######################################################################

# define nan object
NaN = np.nan

# recover appropriate columns
df = df.filter(['Continent', 'Country,Other','TotalCases','TotalRecovered','Serious,Critical','ActiveCases','TotalDeaths']) 

# strip artefacts
df = df.replace(',','', regex=True)
df = df.replace('\+','', regex=True)
df = df.replace('', NaN, regex=True)

# convert appropriate columns to integer type
df[['TotalCases', 'TotalRecovered','Serious,Critical','ActiveCases','TotalDeaths']] = df[['TotalCases', 'TotalRecovered','Serious,Critical','ActiveCases','TotalDeaths']].apply(pd.to_numeric, downcast = 'float', errors = 'coerce')
df.head(20)

# add date of processing to all rows
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y")
df['date_processed'] = dt_string

#rename the columns for consistency
df.rename(columns = {'Continent':'continent','Country,Other':'country','TotalCases':'total_cases','TotalRecovered':'total_recovered','Serious,Critical':'critical_active','ActiveCases':'active_cases','TotalDeaths':'total_deaths'}, inplace=True)


## Task 3: Extract Tables

In [23]:
############# world
world_df = df.loc[df['country'] == "World"]

# select our data for plotting:
world_df = world_df.filter(['total_cases','total_recovered','critical_active','active_cases','total_deaths', 'date_processed'])

# this is the correct way to add a new column from existing columns!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
world_df = world_df.assign(non_critical_active = world_df['active_cases'].astype(float) - world_df['critical_active'].astype(float))
# drop active_cases now
world_df = world_df.drop(columns = ['active_cases']).reset_index(drop=True)

# add in percentage columns
world_df = world_df.assign(total_recovered_percentage = round(((world_df['total_recovered'].astype(float) / world_df['total_cases'].astype(float)) * 100),2))
world_df = world_df.assign(critical_active_percentage = round(((world_df['critical_active'].astype(float) / world_df['total_cases'].astype(float)) * 100),2))
world_df = world_df.assign(total_deaths_percentage = round(((world_df['total_deaths'].astype(float) / world_df['total_cases'].astype(float)) * 100),2))
world_df = world_df.assign(non_critical_active_percentage = round(((world_df['non_critical_active'].astype(float) / world_df['total_cases'].astype(float)) * 100),2))



############ continent 
continents_df = df.iloc[0:5]
continents_df.drop(columns=['country'], inplace=True)

# create a row of data to append to the continents for North America
na = df.loc[df['continent'] == "North America"]
del na["continent"]
na = na.replace('','0', regex=True)
row = []
row.append("North America")

for column in na.columns[1:]:
    na[column] = pd.to_numeric(na[column], errors='coerce')
    row.append(round(na[column].sum()))
row = [row]

continents_df = continents_df.append(pd.DataFrame( row, columns=continents_df.columns), ignore_index = True)

continents_df = continents_df.assign(non_critical_active = continents_df['active_cases'].astype(float) - continents_df['critical_active'].astype(float))
# drop active_cases now
continents_df = continents_df.drop(columns = ['active_cases']).reset_index(drop=True)

# add in percentage columns
continents_df = continents_df.assign(total_recovered_percentage = round(((continents_df['total_recovered'].astype(float) / continents_df['total_cases'].astype(float)) * 100),2))
continents_df = continents_df.assign(critical_active_percentage = round(((continents_df['critical_active'].astype(float) / continents_df['total_cases'].astype(float)) * 100),2))
continents_df = continents_df.assign(total_deaths_percentage = round(((continents_df['total_deaths'].astype(float) / continents_df['total_cases'].astype(float)) * 100),2))
continents_df = continents_df.assign(non_critical_active_percentage = round(((continents_df['non_critical_active'].astype(float) / continents_df['total_cases'].astype(float)) * 100),2))

continents_df.date_processed[continents_df.continent == 'North America'] = dt_string

continents_df = continents_df.sort_values(by=['total_cases'], ascending=False).reset_index(drop=True)

############ country
countries_df = df.loc[7:len(df)]
countries_df = countries_df.reset_index()
countries_df = countries_df.drop(columns = 'index')

countries_df = countries_df.assign(non_critical_active = countries_df['active_cases'].astype(float) - countries_df['critical_active'].astype(float))
countries_df = countries_df.drop(columns = ['active_cases']).reset_index(drop=True)


# add in percentage columns
countries_df = countries_df.assign(total_recovered_percentage = round(((countries_df['total_recovered'].astype(float) / countries_df['total_cases'].astype(float)) * 100),2))
countries_df = countries_df.assign(critical_active_percentage = round(((countries_df['critical_active'].astype(float) / countries_df['total_cases'].astype(float)) * 100),2))
countries_df = countries_df.assign(total_deaths_percentage = round(((countries_df['total_deaths'].astype(float) / countries_df['total_cases'].astype(float)) * 100),2))
countries_df = countries_df.assign(non_critical_active_percentage = round(((countries_df['non_critical_active'].astype(float) / countries_df['total_cases'].astype(float)) * 100),2))

countries_df = countries_df.sort_values(by=['total_cases'], ascending=False).reset_index(drop=True)



,continent,total_cases,total_recovered,critical_active,total_deaths,date_processed,non_critical_active,total_recovered_percentage,critical_active_percentage,total_deaths_percentage,non_critical_active_percentage
0,Asia,59823684.0,56350736.0,30859.0,859476.0,22/07/2021,2582613.0,94.19,0.05,1.44,4.32
1,Europe,50386536.0,46361864.0,6697.0,1123793.0,22/07/2021,2894181.0,92.01,0.01,2.23,5.74
2,North America,41670772.0,34885740.0,13507.0,931702.0,22/07/2021,5839829.0,83.72,0.03,2.24,14.01
3,South America,34797900.0,32299920.0,26851.0,1069595.0,22/07/2021,1401533.0,92.82,0.08,3.07,4.03
4,Africa,6393801.0,5580688.0,4325.0,161301.0,22/07/2021,647487.0,87.28,0.07,2.52,10.13


## Task 4: Create CSVs

In [19]:
# create a folder for the csv data extracts if there isn't aready one using the os module
path = os.getcwd()
directory = 'csv_extracts'

if os.path.isdir(directory):
    print("Extracts directory found at '%s'" % (path+directory))
    path = os.path.join(path, directory)

    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        print("Removing old file '%s' at '%s'" % (filename, file_path))
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
else:
    print('Extracts directory not found, creating csv extracts directory and files...')
    path = os.path.join(path, directory)
    os.mkdir(path)
    try:
        if os.path.isdir(directory):
            print("Directory '% s' successfully created" % directory)
    except Exception as e:
        print("Could not create directory '% s'. Reason: %s" % (directory, e))
        
# create csv files from extracts
now = datetime.now()
dt_string = now.strftime("%d%m%Y")

# world df
csv_name = path + "/world_csv_" + dt_string + ".csv"
print(csv_name)
world_df.to_csv(csv_name)

# continents df
csv_name = path + "/continent_csv_" + dt_string + ".csv"
print(csv_name)
continents_df.to_csv(csv_name)

# countries df
csv_name = path + "/country_csv_" + dt_string + ".csv"
print(csv_name)
countries_df.to_csv(csv_name)


Extracts directory found at 'c:\Users\marja\Documents\Repositories\Personal Code\Python-Projects-and-Challenges\Python Covid Tracker\src\covid-task-schedulercsv_extracts'
Removing old file 'continent_csv_22072021.csv' at 'csv_extracts\continent_csv_22072021.csv'
Removing old file 'country_csv_22072021.csv' at 'csv_extracts\country_csv_22072021.csv'
Removing old file 'world_csv_22072021.csv' at 'csv_extracts\world_csv_22072021.csv'
c:\Users\marja\Documents\Repositories\Personal Code\Python-Projects-and-Challenges\Python Covid Tracker\src\covid-task-scheduler\csv_extracts/world_csv_22072021.csv
c:\Users\marja\Documents\Repositories\Personal Code\Python-Projects-and-Challenges\Python Covid Tracker\src\covid-task-scheduler\csv_extracts/continent_csv_22072021.csv
c:\Users\marja\Documents\Repositories\Personal Code\Python-Projects-and-Challenges\Python Covid Tracker\src\covid-task-scheduler\csv_extracts/country_csv_22072021.csv


## Task 5: Upload data extracts to database

In [20]:
engine = create_engine('sqlite:///../application/site.db', echo=True)
sqlite_connection = engine.connect()

world_df.to_sql("world", sqlite_connection, if_exists='replace')

continents_df.to_sql("continents", sqlite_connection, if_exists='replace')

countries_df.to_sql("countries", sqlite_connection, if_exists='replace')

sqlite_connection.close()


2021-07-22 20:23:27,626 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("world")
2021-07-22 20:23:27,626 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-22 20:23:27,627 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("world")
2021-07-22 20:23:27,627 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-22 20:23:27,628 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-07-22 20:23:27,628 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-22 20:23:27,629 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("world")
2021-07-22 20:23:27,630 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-22 20:23:27,631 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-07-22 20:23:27,631 INFO sqlalchemy.engine.Engine [raw sql] ('world',)
2021-07-22 20:23:27,632 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("world")
2021-07-